In [1]:
import json  
import pandas as pd
import numpy as np 
from pandas.io.json import json_normalize  
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
##JSON paths
PATH_PREV_HEATS = 'data/1/previous_heats_with_properties.json'
PATH_CONSTRAINS = 'data/1/constraints.json'
PATH_PROD_SCHED = 'data/1/production_schedule.json'
PATH_SCRAP_INV = 'data/1/scrap_inventory.json'
PATH_SCRAP_ORD = 'data/1/scrap_orders.json'

In [5]:
def loadjsons(jsonpath, **kwargs):
    if 'table_name' in kwargs:
        with open(jsonpath) as prev_heats:
            prev_heats = json.load(prev_heats)
        prev_heats = json_normalize(data = prev_heats[kwargs["table_name"]])
    else:
        with open(jsonpath) as prev_heats:
            prev_heats = json.load(prev_heats)
        prev_heats = json_normalize(data = prev_heats)
    return prev_heats

In [8]:
prev_heats = loadjsons(PATH_PREV_HEATS)
constriants = loadjsons(PATH_CONSTRAINS, **{'table_name': "scrap_type_constraints_per_heat"})
prod_sched = loadjsons(PATH_PROD_SCHED)
scrap_inv = loadjsons(PATH_SCRAP_INV)
scrap_ord = loadjsons(PATH_SCRAP_ORD)


In [9]:
constriants.head()

,scrap_type,type,weight
0,bushlings,minimum,200
1,pig_iron,minimum,100
2,pig_iron,maximum,300


In [10]:
prod_sched.head()

,heat_seq,heat_id,steel_grade,required_weight,chemistry.cu_pct
0,101,heat-1001,ST1,1000,0.1
1,101,heat-1002,ST1,1000,0.1
2,101,heat-1003,ST1,1000,0.1
3,101,heat-1004,ST1,1000,0.1
4,101,heat-1005,ST1,1000,0.1


In [11]:
scrap_inv.head()

,scrap_type,status,weight
0,bushling,on_hand,5000
1,pig_iron,on_hand,1500
2,municipal_shred,on_hand,12000
3,skulls,on_hand,8000


In [12]:
scrap_ord.head()

,scrap_type,order_date,status,weight,price_per_ton
0,bushling,20191005,delivered,600,75
1,bushling,20191010,delivered,1000,90
2,bushling,20191014,delivered,500,95
3,bushling,20191020,delivered,1000,85
4,bushling,20191022,delivered,1500,95


In [13]:
scrap_prices = scrap_ord.groupby('scrap_type')['price_per_ton'].mean()
scrap_prices.head()

scrap_type
bushling            88.333333
municipal_shred     41.666667
pig_iron           154.166667
skulls              71.666667
Name: price_per_ton, dtype: float64

In [36]:
X = prev_heats[['actual_recipe.bushling','actual_recipe.municipal_shred','actual_recipe.skulls']]
y = prev_heats.loc[:, ['chemistry.cu_pct']]

In [37]:
print(X)
print(y)

    actual_recipe.bushling  actual_recipe.municipal_shred  \
0                      300                            350   
1                      300                            300   
2                      300                            300   
3                      300                            350   
4                      300                            350   
5                      275                            350   
6                      275                            350   
7                      250                            350   
8                      250                            350   
9                      250                            350   
10                     250                            350   
11                     200                            400   
12                     200                            400   
13                     200                            400   
14                     200                            400   

    actual_recipe.skull

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=0)

In [39]:
CUregressor = LinearRegression(fit_intercept=False)
CUregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [40]:
#To retrieve the intercept:
print(CUregressor.intercept_)
#For retrieving the slope:
print(CUregressor.coef_)

print(X.columns)

0.0
[[-0.00014528  0.000163    0.00039166]]
Index(['actual_recipe.bushling', 'actual_recipe.municipal_shred',
       'actual_recipe.skulls'],
      dtype='object')


In [19]:
y_pred = regressor.predict(X_test)

In [20]:
print(y_pred)
print(y_test)

[[0.10322967]
 [0.1052201 ]
 [0.11864354]]
[[0.104]
 [0.114]
 [0.124]]


In [21]:
X = prev_heats[['actual_recipe.bushling','actual_recipe.pig_iron','actual_recipe.municipal_shred','actual_recipe.skulls']]
y = prev_heats.loc[:, ['tap_weight']]
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=10)
regressor = LinearRegression(fit_intercept=False)
regressor.fit(X_train, y_train)
#To retrieve the intercept:
print(regressor.intercept_)
#For retrieving the slope:
print(regressor.coef_)


0.0
[[0.97110227 0.66379098 1.01354129 0.83646812]]


In [22]:
y_pred = regressor.predict(X_test)
print(y_pred)
print(y_test)

[[946.12195122]
 [939.3902439 ]
 [941.51219512]]
[[935]
 [965]
 [955]]


In [23]:
print(X_test)

[[300 200 350 200]
 [250 200 350 250]
 [200 200 400 250]]


In [24]:
from scipy.optimize import curve_fit
import scipy

In [25]:
X_train_scy = np.transpose(X_train)
X_train_scy = np.squeeze(X_train_scy)
X_train_scy

array([[275, 250, 300, 250, 275, 200, 200, 300, 300, 300, 200, 250],
       [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200],
       [350, 350, 300, 350, 350, 400, 400, 300, 350, 350, 400, 350],
       [225, 250, 250, 250, 225, 250, 250, 250, 200, 200, 250, 250]],
      dtype=int64)

In [26]:
y_train_scy = np.transpose(y_train)
y_train_scy = np.squeeze(y_train_scy)
y_train_scy

array([930, 940, 940, 955, 910, 945, 925, 920, 970, 945, 940, 975],
      dtype=int64)

In [27]:
def fn(x, b, c, d, e):
    return b*x[0] + c*x[1] + d*x[2] + e*x[3]

popt, pcov = curve_fit(fn, X_train_scy, y_train_scy, bounds=(0, 1))
print(popt)

[0.90508707 0.94435555 0.9475261  0.77045293]


In [28]:
def yieldmodel(x):
    return 0.90508707*x[0] + 0.94435555*x[1] + 0.947526*x[2] + 0.77045293*x[3]

In [29]:
x_test_scy = np.transpose(X_test)
x_test_scy = np.squeeze(x_test_scy)
x_test_scy

array([[300, 250, 200],
       [200, 200, 200],
       [350, 350, 400],
       [200, 250, 250]], dtype=int64)

In [30]:
yieldmodel(x_test_scy)

array([946.121917 , 939.39021  , 941.5121565])

In [31]:
print(y_test)

[[935]
 [965]
 [955]]


In [43]:
cu_bushling = CUregressor.coef_[0][0]
cu_pig_iron = 0
cu_municipal_shred = CUregressor.coef_[0][1]
cu_skulls = CUregressor.coef_[0][2]

In [44]:
yield_bushling = popt[0]
yield_pig_iron = popt[1]
yield_municipal_shred = popt[2]
yield_skulls = popt[3]

In [45]:
scrap_prices.head()

scrap_type
bushling            88.333333
municipal_shred     41.666667
pig_iron           154.166667
skulls              71.666667
Name: price_per_ton, dtype: float64

In [59]:
some_tap_weight = 990
cu_steel_grade = 0.15

In [65]:
# import PuLP
from pulp import *

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Cost of Scrap in Steel Grade Recipes", LpMinimize)

# Create problem variables
bushlings_wgt=LpVariable("bushlings_wgt",lowBound=200,upBound=None, cat='Continuous')
pig_iron_wgt=LpVariable("pig_iron_wgt",lowBound=100, upBound=300, cat='Continuous')
municipal_shred_wgt=LpVariable("municipal_shred_wgt",lowBound=0,upBound=None, cat='Continuous')
skulls_wgt=LpVariable("skulls_wgt",lowBound=0, upBound=None, cat='Continuous')


# The objective function is added to 'prob' first
prob += scrap_prices.bushling*bushlings_wgt + scrap_prices.pig_iron*pig_iron_wgt + scrap_prices.municipal_shred*municipal_shred_wgt + scrap_prices.skulls*skulls_wgt, "cost; to be minimized"
# The two constraints are entered
prob += yield_bushling*bushlings_wgt + yield_pig_iron*pig_iron_wgt + yield_municipal_shred*municipal_shred_wgt + yield_skulls*skulls_wgt == some_tap_weight, "YIELD constraint"
prob += cu_bushling*bushlings_wgt + cu_pig_iron*pig_iron_wgt + cu_municipal_shred*municipal_shred_wgt + cu_skulls*skulls_wgt <= cu_steel_grade, "Cu constraint"

# The problem data is written to an .lp file
prob.writeLP("Scrap_recipes.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [66]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

bushlings_wgt = 200.0
municipal_shred_wgt = 754.11857
pig_iron_wgt = 100.0
skulls_wgt = 0.0
